In [1]:
from pymatgen.core import Structure, Lattice

s = Structure(Lattice.cubic(3.147), ['Mo', 'Mo'], 
              [[0, 0, 0], [0.5, 0.5, 0.5]])  # a simple Mo bcc lattice

## Compute bispectrum coefficients of structures

In [2]:
from maml.describers import BispectrumCoefficients

bc = BispectrumCoefficients(rcutfac=0.5, twojmax=4,
                            element_profile={'Mo': {'r': 8, 'w': 1}})

In [3]:
bc.transform([s, s*[1, 2, 1]])

0-0-0    1-0-1    1-1-2    2-0-2     2-1-3     2-2-2  \
input_index                                                             
0           0  19521.8  194.323  2.92983  5.91693 -0.050557  0.059473   
            1  19521.8  194.323  2.92983  5.91693 -0.050557  0.059473   
1           0  19521.8  194.323  2.92983  5.91693 -0.050557  0.059473   
            1  19521.8  194.323  2.92983  5.91693 -0.050557  0.059473   
            2  19521.8  194.323  2.92983  5.91693 -0.050557  0.059473   
            3  19521.8  194.323  2.92983  5.91693 -0.050557  0.059473   

                  2-2-4     3-0-3   3-1-4     3-2-3    3-3-4    4-0-4  \
input_index                                                             
0           0 -0.056937  0.065096  0.0363  0.000654 -0.00047  3.25385   
            1 -0.056937  0.065096  0.0363  0.000654 -0.00047  3.25385   
1           0 -0.056937  0.065096  0.0363  0.000654 -0.00047  3.25385   
            1 -0.056937  0.065096  0.0363  0.000654 -0.00047  3.25385   
            2 -0.056937  0.065096  0.0363  0.000654 -0.00047  3.25385   
            3 -0.056937  0.065096  0.0363  0.000654 -0.00047  3.25385   

                  4-2-4     4-4-4  
input_index                        
0           0  0.032704 -0.018786  
            1  0.032704 -0.018786  
1           0  0.032704 -0.018786  
            1  0.032704 -0.018786  
            2  0.032704 -0.018786  
            3  0.032704 -0.018786

In [4]:
s_large = s * [5, 5, 5]

In [11]:
%%timeit -n1 -r1

_ = bc.transform([s_large] * 100)  # time for calculating 100 structures

26 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Parallel computation with progress report

In [6]:
bc_w_parallel = BispectrumCoefficients(rcutfac=0.5, twojmax=4,
                                       element_profile={'Mo': {'r': 8, 'w': 1}},
                                       verbose=True, # show progress
                                       n_jobs=2)  # use 2 CPUs

In [12]:
%%timeit -n1 -r1
_ = bc_w_parallel.transform([s_large] * 100)  # time for calculating 100 structures

100%|██████████| 100/100 [00:17<00:00,  5.75it/s]


17.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## With cache

In [8]:
bc_w_cache = BispectrumCoefficients(rcutfac=0.5, twojmax=4,
                                    element_profile={'Mo': {'r': 8, 'w': 1}},
                                    verbose=True, # show progress
                                    memory=True,  # saved to local
                                    n_jobs=-1)  # use all CPU

INFO:maml.base._describer:Created temporary directory /var/folders/ky/tm77xf4978b75_2bcfpb2lxm0000gn/T/tmp9efue9uw
INFO:maml.base._describer:Using 4 jobs for computation


In [13]:
%%timeit -n1 -r1
_ = bc_w_cache.transform([s_large] * 100)  # time for calculating 100 structures

100%|██████████| 100/100 [00:04<00:00, 20.56it/s]


5.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


We can see that the run time has been reduced substantially. 